# 01 — Build the fact of a Sydney day

This notebook demonstrates how to build a simple regression model for the synthetic Sydney day dataset.

Steps include:

1. Loading the synthetic mart generated via `make ingest && make marts && make analyze`.
2. Performing light feature engineering.
3. Training a linear regression model.
4. Generating predictions with bootstrap confidence intervals.
5. Running a simple sensitivity analysis to understand the impact of feature changes.

💡 **Rebuild the mart before running the notebook**

```bash
make ingest && make marts && make analyze
```

The commands write synthetic raw inputs, rebuild `data/marts/fact_day.parquet`, and refresh supporting quality reports so the analysis always reflects the latest ingest.

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## Load the mart and inspect

In [ ]:
PROJECT_ROOT = Path(os.environ.get('OPAL_OCEAN_PROJECT_ROOT', Path.cwd()))
MARTS_DIR = Path(os.environ.get('OPAL_OCEAN_MARTS_DIR', PROJECT_ROOT / 'data' / 'marts'))
FACT_DAY_PATH = MARTS_DIR / 'fact_day.parquet'
if not FACT_DAY_PATH.exists():
    raise FileNotFoundError('Run `make ingest && make marts && make analyze` to build data/marts/fact_day.parquet.')

df = pd.read_parquet(FACT_DAY_PATH)
df['date'] = pd.to_datetime(df['date'])
if 'beach_ok' in df.columns:
    df['beach_ok'] = df['beach_ok'].astype(int)
df.head()

### Feature overview

In [ ]:
df.describe(include='all')

## Split features and target

In [ ]:
target = 'mood_1_5'
features = [
    'weekday',
    'commute_minutes',
    'opal_cost',
    'reliability',
    'pm25_mean',
    'rain_24h_mm',
    'beach_ok',
    'steps',
    'sleep_hours',
    'caffeine_mg',
]
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train.head()

## Build the modeling pipeline

In [ ]:
categorical_features = ['weekday']
numeric_features = [col for col in features if col not in categorical_features]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features),
    ]
)
model = Pipeline(steps=[('prep', preprocessor), ('regressor', LinearRegression())])
model.fit(X_train, y_train)

## Evaluate on the hold-out set

In [ ]:
y_pred = model.predict(X_test)
print('R^2:', r2_score(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))

## Bootstrap prediction intervals

In [ ]:
def bootstrap_prediction_interval(pipeline, features, target, samples=200, alpha=0.05, random_state=42):
    rng = np.random.default_rng(random_state)
    preds = []
    X = features.to_numpy()
    y = target.to_numpy()
    n = len(features)
    for _ in range(samples):
        idx = rng.integers(0, n, size=n)
        X_sample = features.iloc[idx]
        y_sample = target.iloc[idx]
        pipeline.fit(X_sample, y_sample)
        preds.append(pipeline.predict(features))
    preds = np.vstack(preds)
    lower = np.percentile(preds, 100 * alpha / 2, axis=0)
    upper = np.percentile(preds, 100 * (1 - alpha / 2), axis=0)
    point = pipeline.fit(features, target).predict(features)
    return point, lower, upper

point_pred, lower_ci, upper_ci = bootstrap_prediction_interval(model, X_train, y_train)
intervals = pd.DataFrame({
    'date': df.loc[X_train.index, 'date'],
    'point_pred': point_pred,
    'lower_ci': lower_ci,
    'upper_ci': upper_ci
}).reset_index(drop=True)
intervals.head()

## Sensitivity analysis

We assess how marginal changes in each feature affect the predicted mood score by perturbing one feature at a time.

In [ ]:
baseline = X_train.mean(numeric_only=True).to_dict()
baseline['weekday'] = X_train['weekday'].mode().iat[0]
baseline_df = pd.DataFrame([baseline])
baseline_prediction = model.predict(baseline_df)[0]
results = []
perturbations = {
    'commute_minutes': [-20, 20],
    'reliability': [-0.2, 0.2],
    'pm25_mean': [-5, 5],
    'rain_24h_mm': [-10, 10],
    'steps': [-1500, 1500],
    'sleep_hours': [-1.0, 1.0],
    'caffeine_mg': [-100, 100],
}
for feature, deltas in perturbations.items():
    for delta in deltas:
        scenario = baseline_df.copy()
        if feature == 'reliability':
            scenario[feature] = (scenario[feature] + delta).clip(0.0, 1.0)
        else:
            scenario[feature] = scenario[feature] + delta
        prediction = model.predict(scenario)[0]
        results.append({
            'feature': feature,
            'delta': delta,
            'prediction': prediction,
            'change_vs_baseline': prediction - baseline_prediction
        })
sensitivity_df = pd.DataFrame(results)
sensitivity_df

The sensitivity table shows how each variable shifts the predicted mood score relative to the average training day.